In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from tqdm.notebook import tqdm
import re
import transformers

import os
os.environ['TRANSFORMERS_VERBOSITY'] = 'error'
import logging
transformers.logging.set_verbosity_error()

In [2]:
# deepseek-ai/deepseek-math-7b-instruct
# nvidia/AceMath-7B-Instruct
# nvidia/OpenMath-Nemotron-7B
# Qwen/Qwen2.5-Math-7B-Instruct
# casperhansen/deepseek-r1-distill-qwen-7b-awq !!!

In [3]:
model_name = "nvidia/OpenMath-Nemotron-7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [5]:
def ask_model(messages):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=2 ** 20,
        temperature=0.0
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [6]:
import re

def extract_answer(text):
    """
    Извлекает содержимое из \boxed{} и преобразует:
    - \frac{числитель}{знаменатель} → "числитель/знаменатель"
    - \begin{pmatrix} ... \end{pmatrix} → элементы через ";"
    - Обрабатывает отрицательные числа и знаки минус
    - Обрабатывает множественные точки с координатами
    - Заменяет запятые на | и оборачивает точки в ()
    - Обрабатывает матрицы в виде отдельных элементов через ;
    
    Параметры:
    text (str): Текст с LaTeX-выражением
    
    Возвращает:
    str: Результат в соответствующем формате
    """
    # Паттерн для извлечения содержимого \boxed{}
    boxed_pattern = r'\\boxed\s*{((?:[^{}]|\{[^{}]*\})*)}'
    boxed_match = re.search(boxed_pattern, text, re.DOTALL)
    
    if not boxed_match:
        return None
    
    content = boxed_match.group(1).strip()
    
    # Обработка множественных точек с координатами
    # Проверяем наличие паттерна координат: (x,y) или \left(x,y\right)
    if re.search(r'(?:\\left\s*\(|\().*?,.*?(?:\\right\s*\)|\))', content) and ';' in content:
        # Разбиваем на отдельные точки по точке с запятой
        points = re.split(r';\s*', content)
        points = [p.strip() for p in points if p.strip()]
        
        result_points = []
        for point in points:
            # Обработка отдельных точек вида (x, y), \left(x, y\right), или просто x, y
            point_match = re.search(r'(?:\\left\s*\(|\()?\s*([^,]+)\s*,\s*([^)]+)\s*(?:\\right\s*\)|\))?', point)
            if point_match:
                x = process_coordinate(point_match.group(1).strip())
                y = process_coordinate(point_match.group(2).strip())
                result_points.append(f"({x}|{y})")
        
        if result_points:
            return '; '.join(result_points)
    
    # Обработка матричных элементов через точку с запятой (без координатных скобок)
    elif ';' in content:
        # Разбиваем на отдельные элементы
        elements = re.split(r';\s*', content)
        elements = [elem.strip() for elem in elements if elem.strip()]
        
        result_elements = []
        for element in elements:
            processed_element = process_single_expression(element)
            result_elements.append(processed_element)
        
        if result_elements:
            return ';'.join(result_elements)
    
    # Обработка одиночных координат
    elif re.search(r'(?:\\left\s*\(|\()\s*[^,]+\s*,\s*[^)]+\s*(?:\\right\s*\)|\))', content):
        # Одиночная точка с координатами
        point_match = re.search(r'(?:\\left\s*\(|\()?\s*([^,]+)\s*,\s*([^)]+)\s*(?:\\right\s*\)|\))?', content)
        if point_match:
            x = process_coordinate(point_match.group(1).strip())
            y = process_coordinate(point_match.group(2).strip())
            return f"({x}|{y})"
    
    # Обработка одиночных выражений
    return process_single_expression(content)

def process_coordinate(coord):
    """Обрабатывает отдельную координату (число или дробь)"""
    # Удаляем возможные LaTeX-команды \left и \right
    coord = re.sub(r'\\left|\\right', '', coord).strip()
    
    # Обработка знака
    sign = ''
    if coord.startswith('-'):
        sign = '-'
        coord = coord[1:]
    elif coord.startswith('+'):
        sign = '+'
        coord = coord[1:]
    
    # Обработка дробей
    frac_patterns = [
        r'\\frac\s*{([^{}]+)}\s*{([^{}]+)}',
        r'\\dfrac\s*{([^{}]+)}\s*{([^{}]+)}'
    ]
    
    for pattern in frac_patterns:
        frac_match = re.search(pattern, coord)
        if frac_match:
            numerator = frac_match.group(1).strip()
            denominator = frac_match.group(2).strip()
            return f"{sign}{numerator}/{denominator}"
    
    return sign + coord if sign else coord

def process_single_expression(content):
    """Обрабатывает одиночное выражение"""
    # Удаляем возможные LaTeX-команды \left и \right
    content = re.sub(r'\\left|\\right', '', content).strip()
    
    # Обработка знака
    sign = ''
    if content.startswith('-'):
        sign = '-'
        content = content[1:]
    elif content.startswith('+'):
        sign = '+'
        content = content[1:]
    
    # Обработка дробей
    frac_patterns = [
        r'\\frac\s*{([^{}]+)}\s*{([^{}]+)}',
        r'\\dfrac\s*{([^{}]+)}\s*{([^{}]+)}'
    ]
    
    for pattern in frac_patterns:
        frac_match = re.search(pattern, content)
        if frac_match:
            numerator = frac_match.group(1).strip()
            denominator = frac_match.group(2).strip()
            return f"{sign}{numerator}/{denominator}"
    
    # Обработка матриц
    pmatrix_pattern = r'\\begin\s*{pmatrix}(.*?)\\end\s*{pmatrix}'
    pmatrix_match = re.search(pmatrix_pattern, content, re.DOTALL)
    if pmatrix_match:
        inner_content = pmatrix_match.group(1).strip()
        elements = re.split(r'\\\\', inner_content)
        cleaned_elements = [elem.strip() for elem in elements if elem.strip()]
        return ';'.join(cleaned_elements)
    
    # Если есть запятая в одиночном выражении, заменяем на |
    if ',' in content:
        content = content.replace(',', '|')
    
    return sign + content if sign else content

<>:4: SyntaxWarning: invalid escape sequence '\e'
<>:4: SyntaxWarning: invalid escape sequence '\e'
C:\Users\User\AppData\Local\Temp\ipykernel_24548\457778426.py:4: SyntaxWarning: invalid escape sequence '\e'
  """


In [7]:
SYSTEM_PROMPT = """You are a helpful math expert. You are the most powerful math expert. Please solve the problems with deep reasoning. You are careful and always recheck your conclusions. You will never give the answer directly until you have enough confidence. You should think step-by-step. You need to send me a csv table with problem and answer. Format:
task — text of the problem
answer — list of the answers separated by «;».
Return final answer within \\boxed{}. Like \\boxed{1/3} or \\boxed{0.000}.
You should place all answers in one \\boxed{}. For example:
Answers are 0 and 1. You should return \\boxed{0;1}
"""

In [8]:
# messages = [{"role": "system", "content": SYSTEM_PROMPT}]

In [9]:
translated = pd.read_csv('data/translated.csv')
sub = pd.read_csv('data/anastasia.csv')

translated['answer'] = sub['answer']

In [ ]:
responses = []
answers1 = []
answers2 = []
answers3 = []
data = []

for i, row in tqdm(translated.iterrows(), total=len(translated)):
    task = row.task
    true_answer = row.answer
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    messages.append({
        "role": "user",
        "content": task
    })
    response1 = ask_model(messages)
    response2 = ask_model(messages)
    response3 = ask_model(messages)
    resps = [response1, response2, response3]
    responses.append(resps)
    answer1 = extract_answer(response1[len(response1)-300:])
    answer2 = extract_answer(response2[len(response2)-300:])
    answer3 = extract_answer(response3[len(response3)-300:])

    answers1.append(answer1)
    answers2.append(answer2)
    answers3.append(answer3)

    print(f"{i+1}) True answer: {true_answer}. Predicted answers: {answer1, answer2, answer3}")
    data.append({
        "task": task,
        "true_answer": true_answer,
        "predicted_answer1": answer1,
        "predicted_answer2": answer2,
        "predicted_answer3": answer3,
    })

  0%|          | 0/200 [00:00<?, ?it/s]

1) True answer: -21.000. Predicted answers: ('-21.000', '-21.000', '-21.000')
2) True answer: 4.482. Predicted answers: ('4.482', '4.482', '4.482')
3) True answer: 2.000. Predicted answers: ('2.000', '2.000', '2.000')


In [ ]:
data_df = pd.DataFrame(data)
# data_df['predicted_answer'] = list(map(lambda x: f"[{extract_answer(x[len(x)-200:])}]", responses))
data_df.to_csv('part_4.csv', index=False)

In [ ]:
data_df['predicted_answer'] = data_df['predicted_answer'].apply(lambda x: str(x).replace(',', ';').replace(' ', '')) #.replace('(\\left', '').replace('(\\right', '').replace('\\left', '').replace('\\right', '')

In [ ]:
data_df[data_df.true_answer != data_df.predicted_answer] #[['task', 'true_answer']].rename(columns={'true_answer': 'answer'}).reset_index(drop=True).to_csv('errors.csv', index=False)